<참조 : https://www.tensorflow.org/tutorials/customization/basics?hl=ko>

## 텐서와 연산
  - 필요한 패키지 임포트
  - 텐서(Tensor) 생성 및 사용
  - GPU 가속기 사용
  - tf.data.Dataset 시연

In [2]:
# !pip install -q tensorflow-gpu==2.0.0-rc1
# GPU 없음

In [3]:
import tensorflow as tf

### 텐서 : Tensor
  - 다차원 배열 : Numpy ndarray 객체와 비슷
  - tf.Tensor : 데이터 타입과 크기를 가집니다
    - GPU 가속기 메모리에 상주할 수 있습니다
  - tensor 연산
    - tf.add
    - tf.matmul
    - tf.linalg.inv 등

In [4]:
print(tf.add(1, 2))
print(tf.add([1, 2], [3, 4]))
print(tf.square(5))
print(tf.reduce_sum([1, 2, 3]))

# 연산자 오버로딩(overloading)을 지원합니다
print(tf.square(2) + tf.square(3))

tf.Tensor(3, shape=(), dtype=int32)
tf.Tensor([4 6], shape=(2,), dtype=int32)
tf.Tensor(25, shape=(), dtype=int32)
tf.Tensor(6, shape=(), dtype=int32)
tf.Tensor(13, shape=(), dtype=int32)


In [6]:
x = tf.matmul([[1]], [[2, 3]])
print(x)
print(x.shape)
print(x.dtype)

tf.Tensor([[2 3]], shape=(1, 2), dtype=int32)
(1, 2)
<dtype: 'int32'>


### Numpy ndarray와 tf.Tensor의 차이
  - 텐서는 가속기 메모리(GPU, TPU 등)에서 사용할 수 있습니다
  - 텐서는 불변성(immutable)을 가집니다  
    
### Numpy와 호환성
  - 텐서플로우 연산은 자동으로 넘파이 배열을 텐서로 변환합니다
  - 넘파이 연산은 자동으로 텐서를 넘파이 배열로 변환합니다  
    
  - 텐서는 .numpy() 메서드를 호출하여 넘파이 배열로 변환할 수 있습니다

In [7]:
import numpy as np

ndarray = np.ones([3, 3])

print('텐서플로 연산은 자동적으로 넘파이 배열을 텐서로 변환합니다.')
tensor = tf.multiply(ndarray, 42)
print(tensor)

print('그리고 넘파이 연산은 자동적으로 텐서를 넘파이 배열로 변환합니다.')
print(np.add(tensor, 1))

print('.numpy() 메서드는 텐서를 넘파이 배열로 변환합니다.')
print(tensor.numpy())

텐서플로 연산은 자동적으로 넘파이 배열을 텐서로 변환합니다.
tf.Tensor(
[[42. 42. 42.]
 [42. 42. 42.]
 [42. 42. 42.]], shape=(3, 3), dtype=float64)
그리고 넘파이 연산은 자동적으로 텐서를 넘파이 배열로 변환합니다.
[[43. 43. 43.]
 [43. 43. 43.]
 [43. 43. 43.]]
.numpy() 메서드는 텐서를 넘파이 배열로 변환합니다.
[[42. 42. 42.]
 [42. 42. 42.]
 [42. 42. 42.]]


### GPU 가속

In [8]:
x = tf.random.uniform([3, 3])

print('GPU 사용이 가능한가: ')
print(tf.test.is_gpu_available())

print('텐서가 GPU #0에 있는가: ')
print(x.device.endswith('GPU:0'))

GPU 사용이 가능한가: 
Instructions for updating:
Use `tf.config.list_physical_devices('GPU')` instead.
False
텐서가 GPU #0에 있는가: 
False


In [9]:
import time

def time_matmul(x):
    start = time.time()
    for loop in range(10):
        tf.matmul(x, x)
        
    result = time.time() - start
    
    print('10 loops: {:0.2f}ms'.format(1000*result))
    
# CPU에서 강제 실행합니다
print('On CPU: ')
with tf.device('CPU:0'):
    x = tf.random.uniform([1000, 1000])
    assert x.device.endswith('CPU:0')
    time_matmul(x)
    
# GPU #0가 이용가능시 GPU #0에서 강제 실행합니다.
if tf.test.is_gpu_available():
    print('On GPU: ')
    with tf.device('GPU:0'):    # Or GPU:1 for the 2nd GPU, GPU:2 for the 3rd etc.
        x = tf.random.uniform([1000, 1000])
        assert x.device.endswith('GPU:0')
        time_matmul(x)

On CPU: 
10 loops: 1640.86ms


### 데이터세트

In [12]:
ds_tensors = tf.data.Dataset.from_tensor_slices([1, 2, 3, 4, 5, 6])

# csv 파일을 생성합니다
import tempfile
_, filename = tempfile.mkstemp()

with open(filename, 'w') as f:
    f.write('''Line 1
    Line 2
    Line 3
    ''')
    
ds_file = tf.data.TextLineDataset(filename)

#### 변환 적용
  - 맵(map), 배치(batch), 셔플(shuffle)과 같은 변환 함수를 사용하여 데이터세트의 레코드에 적용

In [13]:
ds_tensors = ds_tensors.map(tf.square).shuffle(2).batch(2)

ds_file = ds_file.batch(2)
ds_file

<BatchDataset shapes: (None,), types: tf.string>

#### 반복

In [14]:
print('ds_tensors 요소: ')
for x in ds_tensors:
    print(x)
    
print('\nds_file 요소: ')
for x in ds_file:
    print(x)

ds_tensors 요소: 
tf.Tensor([1 4], shape=(2,), dtype=int32)
tf.Tensor([ 9 16], shape=(2,), dtype=int32)
tf.Tensor([36 25], shape=(2,), dtype=int32)

ds_file 요소: 
tf.Tensor([b'Line 1' b'    Line 2'], shape=(2,), dtype=string)
tf.Tensor([b'    Line 3' b'    '], shape=(2,), dtype=string)
